# Benchmark 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import numpy as np
import scanpy as sc
import time
from pathlib import Path
import torch
import concord as ccd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib as mpl

from matplotlib import font_manager, rcParams
custom_rc = {
    'font.family': 'Arial',  # Set the desired font for this plot
}

mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.fonttype'] = 42

/opt/anaconda3/envs/concord/lib/python3.10/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [3]:
proj_name = "pbmc_Darmanis"
save_dir = f"../save/{proj_name}-{time.strftime('%b%d')}/"
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)

data_dir = f"../data/{proj_name}/"
data_dir = Path(data_dir)
data_dir.mkdir(parents=True, exist_ok=True)
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(device)
seed = 0
ccd.ul.set_seed(seed)

file_suffix = f"{time.strftime('%b%d-%H%M')}"
file_suffix

cpu


'Jun09-2244'

# Download CellxGene AnnData Objects

In [4]:
##USER Defined Anndata Object Directory
##Keep this if you want to use immediately with other notebooks
ANNDATA_OBJECT_DIR='../data/pbmc_Darmanis/'


In [6]:
##Define domain names for cellxgene
domain_name = "cellxgene.cziscience.com"
site_url = f"https://{domain_name}"
api_url_base = f"https://api.{domain_name}"

##Define specific collection ID for this study
collection_id = "398e34a9-8736-4b27-a9a7-31a47a67f446"

##Fetch collection
collection_path = f"/curation/v1/collections/{collection_id}"
collection_url = f"{api_url_base}{collection_path}"
res = requests.get(url=collection_url)
res.raise_for_status()
res_content = res.json()

In [7]:
kits_downloaded = []
for dataset in res_content['datasets']:
    assets = dataset["assets"]
    dataset_id = dataset["dataset_id"]
    kit_name = dataset['title']
    kits_downloaded.append(kit_name)
    for asset in assets:
        if asset['filetype'] == 'H5AD':
            download_filename = os.path.join(ANNDATA_OBJECT_DIR, f'{kit_name}_annotated.h5ad')
            print(f"\nDownloading {kit_name} to {download_filename} ... ")
            with requests.get(asset["url"], stream=True) as res:
                res.raise_for_status()
                filesize = int(res.headers["Content-Length"])
                with open(download_filename, "wb") as df:
                    total_bytes_received = 0
                    for chunk in res.iter_content(chunk_size=1024 * 1024):
                        df.write(chunk)
                        total_bytes_received += len(chunk)
                        percent_of_total_upload = float("{:.1f}".format(total_bytes_received / filesize * 100))
                        color = "\033[38;5;10m" if percent_of_total_upload == 100 else ""
                        print(f"\033[1m{color}{percent_of_total_upload}% downloaded\033[0m\r", end="")
    print("\n\nDone downloading assets")


100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets

100.0% downloaded

Done downloading assets


In [8]:
kits_downloaded = sorted(kits_downloaded)
print(f'Downloaded Kit Data: {kits_downloaded}')


Downloaded Kit Data: ['10X_3-rep1', '10X_3-rep2', '10X_5-rep1', '10X_5-rep2', '10X_FRP-rep1', '10X_FRP-rep2', 'BD-rep1', 'BD-rep2', 'Fluent-rep1', 'Fluent-rep2', 'Fluent-rep3', 'Honeycomb-rep1', 'Honeycomb-rep2', 'Parse-rep1', 'Scale-rep1', 'Scipio-rep1', 'Scipio-rep2', 'harmony_integrated_data']


In [ ]:
# concatenate all anndata objects except the harmony_integrated_data_annotated.h5ad
anndata_objects = [f for f in Path(ANNDATA_OBJECT_DIR).glob('*.h5ad') if 'harmony_integrated_data_annotated.h5ad' not in str(f)]
print(f'Anndata Objects: {anndata_objects}')
# concatenate all anndata objects
adata_list = [sc.read_h5ad(f) for f in anndata_objects]
adata = adata_list[0].concatenate(adata_list[1:], batch_key='dataset', batch_categories=[f.stem for f in anndata_objects])
# save the concatenated anndata object
adata.write_h5ad(os.path.join(ANNDATA_OBJECT_DIR, 'concatenated_annotated.h5ad'))
print(f'Concatenated Anndata Object saved to {ANNDATA_OBJECT_DIR}/concatenated_annotated.h5ad')

****

downsample the anndata objects

In [9]:
# Define the directory for the anndata object
ANNDATA_OBJECT_DIR = "../data/pbmc_Darmanis/"

In [10]:
# downsample each dataset to 500 cells and concatenate them into a new anndata object
downsampled_adata_list = []
# concatenate all anndata objects except the downsampled_concatenated_annotated.h5ad
anndata_objects = [f for f in Path(ANNDATA_OBJECT_DIR).glob('*.h5ad') if 'downsampled_' not in str(f)]
for f in anndata_objects:
    adata = sc.read_h5ad(f)
    # downsample to 500 cells
    if adata.n_obs > 500:
        adata = adata[np.random.choice(adata.n_obs, 500, replace=False)]
    downsampled_adata_list.append(adata)
# concatenate all downsampled anndata objects
downsampled_adata = downsampled_adata_list[0].concatenate(downsampled_adata_list[1:], batch_key='dataset', batch_categories=[f.stem for f in anndata_objects])
# save the downsampled anndata object
downsampled_adata.write_h5ad(os.path.join(ANNDATA_OBJECT_DIR, 'downsampled_9K.h5ad'))

# 

----

In [4]:
# load the downsampled anndata object
adata = sc.read_h5ad('../data/pbmc_Darmanis/downsampled_9K.h5ad')

----

In [6]:
# copy the raw counts to a new layer
adata.layers['counts'] = adata.raw.X.copy()

In [5]:
batch_key = 'dataset'
state_key = 'cell_type'

In [6]:
combined_keys = [
        # "unintegrated",
        # "scanorama", "liger", "harmony",
        # "scvi", "scanvi",
        "concord",
        "concord_class", 
        "concord_decoder", "contrastive"
    ]

In [7]:
time_log, ram_log, vram_log = ccd.ul.run_integration_methods_pipeline(
    adata=adata,                          # Your input AnnData object
    methods=combined_keys,            # List of methods to run
    batch_key=batch_key,                    # Column in adata.obs for batch info
    count_layer="counts",                 # Layer name containing raw counts
    class_key=state_key,               # Column in adata.obs for class labels (used in SCANVI and CONCORD variants)
    latent_dim=30,                        # Latent dimensionality for PCA and embeddings
    device='cpu',                        # Or "cpu", or "mps" for Apple Silicon
    return_corrected=False,                   # Whether to store corrected expression matrices
    transform_batch=None,                 # Optionally specify a batch to transform to in scVI
    seed=42,                              # Random seed for reproducibility
    compute_umap=True,                    # Run UMAP for all output embeddings
    umap_n_components=2,
    umap_n_neighbors=30,
    umap_min_dist=0.5,
    verbose=True,                        # Print progress messages
)

Concord - WARNING - save_dir is None. Model and log files will not be saved.
Concord - WARNING - No input feature list provided. It is recommended to first select features using the command `concord.ul.select_features()`.


FAISS not found. Using sklearn for k-NN computation.


p_intra_knn: 0.3


Epoch 0 Training: 119it [00:04, 27.04it/s, loss=3.57]
Epoch 9 Training: 100%|██████████| 119/119 [00:03<00:00, 38.94it/s, loss=3.39]

Concord - WARNING - save_dir is None. Skipping model/config saving.


...

FAISS not found. Using sklearn for k-NN computation.
You specified p_intra_domain as 0.95 but you only have one domain. Resetting p_intra_domain to 1.0.


p_intra_knn: 0.3


Epoch 0 Training: 132it [00:03, 34.67it/s, loss=3.48]
Epoch 9 Training: 100%|██████████| 132/132 [00:04<00:00, 28.09it/s, loss=3.26]

Concord - WARNING - save_dir is None. Skipping model/config saving.



contrastive completed in 43.79 sec.
Running UMAP on contrastive...
✅ Selected methods completed.


In [9]:
# save the adata object
adata.write_h5ad('../data/pbmc_Darmanis/downsampled_9K.h5ad')

In [1]:
# test